This notebook showcases the algorithm for clustering molecular features possibly originating from the same metabolite.

### Installation

First, you need to install the amp package, which contains all the functions we need. The amp package is developed at UEF and contains many useful functions for LC-MS data preprocessing and analysis. Installing amp also installs all the dependencies. Note that an R version 3.5.0 or later is recommended!

You can install amp by running the following code. (For some reason, jupyter does not allow me to install packages but throws a weird error. If this happens, open another R console and install the packages from there.)

In [ ]:
# if (!requireNamespace("devtools", quietly = TRUE)) {
#  install.packages("devtools")
# }
# devtools::install_github("antonvsdata/amp")

NOTE: If this gives you a weird error, run the following line and try again (ref: https://github.com/r-lib/devtools/issues/1900)

In [ ]:
# devtools::install_github("r-lib/remotes", ref = "e56a41e1d0cad55cbe7d60b274b99ab7b7a76b5c")

### Loading data

First, let's load the required package and functions. 

In [1]:
library(amp)
library(openxlsx) # For reading Excel spreadsheets

Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.5.2"Loading required package: magrittr
Warning message:
"package 'magrittr' was built under R version 3.5.2"Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
  

Next, we read in the example data from the file HILIC_NEG_TEST.xlsx:

In [2]:
data_file <- "HILIC_NEG_TEST.xlsx"
abundances <- read.xlsx(data_file, sheet = 1)
feature_info <- read.xlsx(data_file, sheet = 2)

Let's take a peak at the data:

In [3]:
# Dimensions
cat(paste("Abundances:", nrow(abundances), "rows and", ncol(abundances), "columns"))
abundances[1:10, 1:4]

cat(paste("Feature info:", nrow(feature_info), "rows and", ncol(feature_info), "columns"))
feature_info[1:10, ]

Abundances: 276 rows and 538 columns

COMPOUND_174_1115a6_947448,COMPOUND_175_0955a6_3451543,COMPOUND_527_3258a1_2846748,COMPOUND_565_3398a1_2073423
12982,1,12556,286969
18192,19866,24016,594251
14317,19033,1,197562
24180,30353,1,284768
15218,21622,1,661544
15539,10267,11466,238440
18608,18156,13102,350136
12379,29315,1,218183
14054,24522,12189,378187
12316,15498,17065,297489


Feature info: 538 rows and 9 columns

Name,column,mode,mz,mzmin,mzmax,rt,rtmin,rtmax
COMPOUND_174_1115a6_947448,hilic,neg,174.1115,174.1115,174.1115,6.9474480,6.9474480,6.9474480
COMPOUND_175_0955a6_3451543,hilic,neg,175.0955,175.0955,175.0955,6.3451543,6.3451543,6.3451543
COMPOUND_527_3258a1_2846748,hilic,neg,527.3258,527.3258,527.3258,1.2846748,1.2846748,1.2846748
COMPOUND_565_3398a1_2073423,hilic,neg,565.3398,565.3398,565.3398,1.2073423,1.2073423,1.2073423
COMPOUND_555_3106a1_0777795,hilic,neg,555.3106,555.3106,555.3106,1.0777795,1.0777795,1.0777795
COMPOUND_179_0585a0_980333,hilic,neg,179.0585,179.0585,179.0585,0.9803330,0.9803330,0.9803330
COMPOUND_565_3384a1_0822716,hilic,neg,565.3384,565.3384,565.3384,1.0822716,1.0822716,1.0822716
COMPOUND_90_032a1_3770405,hilic,neg,90.0320,90.0320,90.0320,1.3770405,1.3770405,1.3770405
COMPOUND_118_0628a0_7650406,hilic,neg,118.0628,118.0628,118.0628,0.7650406,0.7650406,0.7650406
COMPOUND_558_4648a0_42406148,hilic,neg,558.4648,558.4648,558.4648,0.4240615,0.4240615,0.4240615


### Finding connections

The first step of the algorithm is to find all the connections between features. This means the features that have a large Pearson correlation coefficient and a small retention time difference. Here we use a limit of 0.9 for the correlation coefficient and a 1 second retention time window (the retention times in our data are recorded as minutes).

Note that you can get more information about these functions by running ?function_name

In [4]:
# Uncomment below to view help file
# ?find_connections

Running the function will result in a warning telling we have "no parallel backend registered". This warning can be ignored, but with larger datasets it is possible to parallelize the computation by running:

```
library(doParallel)
registerDoParallel() # Specify number of cores
```

In [5]:
conn <- find_connections(data = abundances, features = feature_info, corr_thresh = 0.9, rt_window = 1/60,
                        name_col = "Name", mz_col = "mz", rt_col = "rt")

Warning message:
"executing %dopar% sequentially: no parallel backend registered"

[1] 100
[1] 200
[1] 300
[1] 400
[1] 500


Let's look at what we got:

In [6]:
dim(conn)
head(conn)

[1] 1274    5

x,y,cor,rt_diff,mz_diff
COMPOUND_179_0585a0_980333,COMPOUND_135_0685a0_97990537,0.9462173,-0.00042763,-43.9900
COMPOUND_179_0585a0_980333,COMPOUND_215_0349a0_9814668,0.9530531,0.00113380,35.9764
COMPOUND_558_4648a0_42406148,COMPOUND_556_4505a0_42484784,0.9859041,0.00078636,-2.0143
COMPOUND_558_4648a0_42406148,COMPOUND_530_434a0_42711142,0.9263948,0.00304994,-28.0308
COMPOUND_558_4648a0_42406148,COMPOUND_278_2246a0_42523193,0.9853918,0.00117045,-280.2402
COMPOUND_558_4648a0_42406148,COMPOUND_280_2401a0_4265691,0.9075599,0.00250762,-278.2247


### Finding dense clusters

So we have 1274 connections between the features. The next step is to construct an undirected graph, split it into connected components and start trimming those components until we only have densely clustered groups of features left. This is achived by setting a threshold for the relative degree. Here we use a threshold of 0.8, meaning that each feature has to be linked to at least  80% of other features in a cluster. Until this condition is satisfied, the feature with the least amount of connections is dropped (the sum of the correlation coefficients is used as a tie-breaker).

After each iteration, the function reports the number of components left.

In [7]:
clusters <- find_clusters(connections = conn, d_thresh = 0.8)

28 components found

14 components found

9 components found

1 components found



This returns a list of clusters, each element is its own list with a vector of feature names and a graph object.

Let's take a look at the number of features in each cluster:

In [8]:
sort(sapply(clusters, function(x) {length(x$features)}), decreasing = TRUE)

[1] 37 15 10  6  5  5  4  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2  2
[26]  2  2  2  2  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
[51]  1  1

As we can see, most of the clusters have a reasonable number of features, but some clusters tend to group together a large number of features. In this case, no data cleaning was applied, but this seems to happen on cleaned data as well when the number of features is large enough.

### Processing clusters

After we have our dense clusters, we can assign a cluster ID to all features. This cluster ID can then be used in the metabolite identification phase, to help the researcher to find features that might represent the same metabolite.

The cluster ID is the name of the feature with the largest median peak area (MPA). If a column named 'MPA' is not present in the feature information table, it is computed and added. For each cluster, the features in that cluster are listed.

In [9]:
feature_info_clustered <- assign_cluster_id(data = abundances, clusters = clusters, features = feature_info, name_col = "Name")

head(feature_info_clustered, 10)

Name,column,mode,mz,mzmin,mzmax,rt,rtmin,rtmax,MPA,Cluster_ID,Cluster_features,Cluster_size
COMPOUND_174_1115a6_947448,hilic,neg,174.1115,174.1115,174.1115,6.9474480,6.9474480,6.9474480,15587.0,COMPOUND_174_1115a6_947448,COMPOUND_174_1115a6_947448,1
COMPOUND_175_0955a6_3451543,hilic,neg,175.0955,175.0955,175.0955,6.3451543,6.3451543,6.3451543,18601.5,COMPOUND_175_0955a6_3451543,COMPOUND_175_0955a6_3451543,1
COMPOUND_527_3258a1_2846748,hilic,neg,527.3258,527.3258,527.3258,1.2846748,1.2846748,1.2846748,1.0,COMPOUND_527_3258a1_2846748,COMPOUND_527_3258a1_2846748,1
COMPOUND_565_3398a1_2073423,hilic,neg,565.3398,565.3398,565.3398,1.2073423,1.2073423,1.2073423,307065.0,COMPOUND_565_3398a1_2073423,COMPOUND_565_3398a1_2073423,1
COMPOUND_555_3106a1_0777795,hilic,neg,555.3106,555.3106,555.3106,1.0777795,1.0777795,1.0777795,16589.5,COMPOUND_555_3106a1_0777795,COMPOUND_555_3106a1_0777795,1
COMPOUND_179_0585a0_980333,hilic,neg,179.0585,179.0585,179.0585,0.9803330,0.9803330,0.9803330,345901.5,Cluster_COMPOUND_179_0585a0_980333,COMPOUND_179_0585a0_980333;COMPOUND_215_0349a0_9814668,2
COMPOUND_565_3384a1_0822716,hilic,neg,565.3384,565.3384,565.3384,1.0822716,1.0822716,1.0822716,79907.0,COMPOUND_565_3384a1_0822716,COMPOUND_565_3384a1_0822716,1
COMPOUND_90_032a1_3770405,hilic,neg,90.0320,90.0320,90.0320,1.3770405,1.3770405,1.3770405,120834.0,COMPOUND_90_032a1_3770405,COMPOUND_90_032a1_3770405,1
COMPOUND_118_0628a0_7650406,hilic,neg,118.0628,118.0628,118.0628,0.7650406,0.7650406,0.7650406,153554.5,COMPOUND_118_0628a0_7650406,COMPOUND_118_0628a0_7650406,1
COMPOUND_558_4648a0_42406148,hilic,neg,558.4648,558.4648,558.4648,0.4240615,0.4240615,0.4240615,154766.5,Cluster_COMPOUND_558_4648a0_42406148,COMPOUND_278_2246a0_42523193;COMPOUND_528_418a0_42738697;COMPOUND_530_434a0_42711142;COMPOUND_556_4505a0_42484784;COMPOUND_558_4648a0_42406148,5


Note that the above function _does not remove any features_ but simple labels features in the same cluster with the same cluster ID. It is also possible to only keep one feature per cluster (the feature with the largest median peak area):

In [10]:
pulled <- pull_clusters(data = abundances, features = feature_info_clustered, name_col = "Name")
cluster_abundances <- pulled$cdata
cluster_feature_info <- pulled$cfeatures

# Dimensions
cat(paste("Abundances:", nrow(cluster_abundances), "rows and", ncol(cluster_abundances), "columns"))
cluster_abundances[1:10, 1:4]

cat(paste("Feature info:", nrow(cluster_feature_info), "rows and", ncol(cluster_feature_info), "columns"))
cluster_feature_info[1:10, ]

Abundances: 276 rows and 430 columns

COMPOUND_174_1115a6_947448,COMPOUND_175_0955a6_3451543,COMPOUND_527_3258a1_2846748,COMPOUND_565_3398a1_2073423
12982,1,12556,286969
18192,19866,24016,594251
14317,19033,1,197562
24180,30353,1,284768
15218,21622,1,661544
15539,10267,11466,238440
18608,18156,13102,350136
12379,29315,1,218183
14054,24522,12189,378187
12316,15498,17065,297489


Feature info: 430 rows and 13 columns

Name,column,mode,mz,mzmin,mzmax,rt,rtmin,rtmax,MPA,Cluster_ID,Cluster_features,Cluster_size
COMPOUND_174_1115a6_947448,hilic,neg,174.1115,174.1115,174.1115,6.9474480,6.9474480,6.9474480,15587.0,COMPOUND_174_1115a6_947448,COMPOUND_174_1115a6_947448,1
COMPOUND_175_0955a6_3451543,hilic,neg,175.0955,175.0955,175.0955,6.3451543,6.3451543,6.3451543,18601.5,COMPOUND_175_0955a6_3451543,COMPOUND_175_0955a6_3451543,1
COMPOUND_527_3258a1_2846748,hilic,neg,527.3258,527.3258,527.3258,1.2846748,1.2846748,1.2846748,1.0,COMPOUND_527_3258a1_2846748,COMPOUND_527_3258a1_2846748,1
COMPOUND_565_3398a1_2073423,hilic,neg,565.3398,565.3398,565.3398,1.2073423,1.2073423,1.2073423,307065.0,COMPOUND_565_3398a1_2073423,COMPOUND_565_3398a1_2073423,1
COMPOUND_555_3106a1_0777795,hilic,neg,555.3106,555.3106,555.3106,1.0777795,1.0777795,1.0777795,16589.5,COMPOUND_555_3106a1_0777795,COMPOUND_555_3106a1_0777795,1
COMPOUND_179_0585a0_980333,hilic,neg,179.0585,179.0585,179.0585,0.9803330,0.9803330,0.9803330,345901.5,Cluster_COMPOUND_179_0585a0_980333,COMPOUND_179_0585a0_980333;COMPOUND_215_0349a0_9814668,2
COMPOUND_565_3384a1_0822716,hilic,neg,565.3384,565.3384,565.3384,1.0822716,1.0822716,1.0822716,79907.0,COMPOUND_565_3384a1_0822716,COMPOUND_565_3384a1_0822716,1
COMPOUND_90_032a1_3770405,hilic,neg,90.0320,90.0320,90.0320,1.3770405,1.3770405,1.3770405,120834.0,COMPOUND_90_032a1_3770405,COMPOUND_90_032a1_3770405,1
COMPOUND_118_0628a0_7650406,hilic,neg,118.0628,118.0628,118.0628,0.7650406,0.7650406,0.7650406,153554.5,COMPOUND_118_0628a0_7650406,COMPOUND_118_0628a0_7650406,1
COMPOUND_558_4648a0_42406148,hilic,neg,558.4648,558.4648,558.4648,0.4240615,0.4240615,0.4240615,154766.5,Cluster_COMPOUND_558_4648a0_42406148,COMPOUND_278_2246a0_42523193;COMPOUND_528_418a0_42738697;COMPOUND_530_434a0_42711142;COMPOUND_556_4505a0_42484784;COMPOUND_558_4648a0_42406148,5


#### Visualization

The package offers basic visualizations of each cluster. The visualizations include

- A heatmap of the m/z differences between the features
- A plot of the median peak area of each feature as a function of the m/z
- m/z as a function of retention time, including the retention time window around each feature
- A visualization of the graph, with node size proportional to the median peak area of the feature, and color indicating degree (darker blue = larger degree).

To visualize the features in each cluster, use:

In [12]:
visualize_clusters(data = abundances, features = feature_info_clustered, clusters = clusters, min_size = 3, rt_window = 1/60,
                   name_col = "Name", mz_col = "mz", rt_col = "rt",
                   file_path = "figures/")